In [4]:
from functions import *
from Bio import AlignIO

In [9]:
query = 'B3GWA1'

In [10]:
alig = readAlig('alignOurs.fa', 'fasta')

In [11]:
lSeq = len(alig[query])
nSeq = len(alig)

In [12]:
alp=list("ACDEFGHIKLMNPQRSTVWY") # this is the AA alphabet
alp
listSubs = []
for i in range(len(alp)):
    for j in range(i+1, len(alp)):
        listSubs.append((alp[i], alp[j]))

listSubs = tuple(listSubs)

In [13]:
listSubs

(('A', 'C'),
 ('A', 'D'),
 ('A', 'E'),
 ('A', 'F'),
 ('A', 'G'),
 ('A', 'H'),
 ('A', 'I'),
 ('A', 'K'),
 ('A', 'L'),
 ('A', 'M'),
 ('A', 'N'),
 ('A', 'P'),
 ('A', 'Q'),
 ('A', 'R'),
 ('A', 'S'),
 ('A', 'T'),
 ('A', 'V'),
 ('A', 'W'),
 ('A', 'Y'),
 ('C', 'D'),
 ('C', 'E'),
 ('C', 'F'),
 ('C', 'G'),
 ('C', 'H'),
 ('C', 'I'),
 ('C', 'K'),
 ('C', 'L'),
 ('C', 'M'),
 ('C', 'N'),
 ('C', 'P'),
 ('C', 'Q'),
 ('C', 'R'),
 ('C', 'S'),
 ('C', 'T'),
 ('C', 'V'),
 ('C', 'W'),
 ('C', 'Y'),
 ('D', 'E'),
 ('D', 'F'),
 ('D', 'G'),
 ('D', 'H'),
 ('D', 'I'),
 ('D', 'K'),
 ('D', 'L'),
 ('D', 'M'),
 ('D', 'N'),
 ('D', 'P'),
 ('D', 'Q'),
 ('D', 'R'),
 ('D', 'S'),
 ('D', 'T'),
 ('D', 'V'),
 ('D', 'W'),
 ('D', 'Y'),
 ('E', 'F'),
 ('E', 'G'),
 ('E', 'H'),
 ('E', 'I'),
 ('E', 'K'),
 ('E', 'L'),
 ('E', 'M'),
 ('E', 'N'),
 ('E', 'P'),
 ('E', 'Q'),
 ('E', 'R'),
 ('E', 'S'),
 ('E', 'T'),
 ('E', 'V'),
 ('E', 'W'),
 ('E', 'Y'),
 ('F', 'G'),
 ('F', 'H'),
 ('F', 'I'),
 ('F', 'K'),
 ('F', 'L'),
 ('F', 'M'),
 ('F', 'N'),